In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('../../data/train.csv')
# Drop column with mostly empty values.
train = train.drop(['VAR_0205', 'VAR_0206', 'VAR_0207', 'VAR_0208', 'VAR_0209', 'VAR_0213', 'VAR_0840'], 1)
# Drop date columns for now...
train = train.drop(['VAR_0073', 'VAR_0075', 'VAR_0156', 'VAR_0157', 'VAR_0158', \
                    'VAR_0159', 'VAR_0166', 'VAR_0167', 'VAR_0168', 'VAR_0169', \
                    'VAR_0176', 'VAR_0177', 'VAR_0178', 'VAR_0179', 'VAR_0204', 'VAR_0217'], 1)
# Drop ID and target column for now...
target = train['target']
train = train.drop(['ID', 'target'], 1)
# feature = [x for x in train.columns if x not in ['ID', 'target']]

/Library/Python/2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# If 'int64' > 20 unique values then it's probably a numerical feature
criteria = train.dtypes == 'int64'
int_train = train[criteria.index[criteria]]
criteria_num = int_train.apply(lambda x: len(x.unique()) > 20)
criteria_cat = int_train.apply(lambda x: len(x.unique()) <= 20)

# Making numeric/categorical dataframes from int64
int_num_train = int_train[criteria_num.index[criteria_num]]
int_cat_train = int_train[criteria_cat.index[criteria_cat]]
print int_train.info()
print '\n', int_num_train.info()
print '\n', int_cat_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145231 entries, 0 to 145230
Columns: 1404 entries, VAR_0002 to VAR_1933
dtypes: int64(1404)
memory usage: 1.5 GB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145231 entries, 0 to 145230
Columns: 995 entries, VAR_0002 to VAR_1933
dtypes: int64(995)
memory usage: 1.1 GB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145231 entries, 0 to 145230
Columns: 409 entries, VAR_0532 to VAR_1931
dtypes: int64(409)
memory usage: 453.2 MB
None


In [4]:
# %%timeit
# numeric_features = train._get_numeric_data()
criteria = train.dtypes == 'float64'
num_train = train[criteria.index[criteria]]
print 'Before concatenating df: ', num_train.shape
num_train = pd.concat([num_train, int_num_train], axis=1)
print 'After concatenating df: ', num_train.shape
filled_mean = numericFeatures.fillna(numeric_features.mean())
# More efficient way to fill in missing values with mean() for numeric features

(145231, 470)
(145231, 1465)


In [39]:
print numericFeatures.isnull().values.sum()
print filledMean.isnull().values.sum()

634179
0


In [40]:
# %%timeit
criteria = train.dtypes == 'object'
cat_train = train[criteria.index[criteria]]
print 'Before concatenating df: ', cat_train.shape
cat_train = pd.concat([cat_train, cat_num_train], axis=1)
print 'After concatenating df: ', cat_train.shape
filled_cat_train = cat_train.apply(lambda x:x.fillna(x.value_counts().index[0]))
# More efficient way to fill in missing values with most frequent value for categorical features

1 loop, best of 3: 695 ms per loop


In [41]:
print cat_train.isnull().values.sum()
print filled_cat_train.isnull().values.sum()

157303
0


In [3]:
num_feat = train.dtypes[train.dtypes == 'float64'].index.values.tolist()
cat_feat = train.dtypes[train.dtypes == 'object'].index.values.tolist()
int_feat = train.dtypes[train.dtypes == 'int64'].index.values.tolist()
print len(num_feat), len(cat_feat)
print len(int_feat)

470 35
1404


In [8]:
# A function that fills in missing values with most frequent value, median, or custom value.
def feature_process(process_func='None', most_frequent_func=False, median_func=False, col_name=[]):
    print 'processing features...'
    for column in col_name:
        print column
        if median_func:
            process_func = train[column].median()
        if most_frequent_func:
            process_func = train[column].value_counts().index.values[0]
        train.loc[train[column].isnull(), column] = process_func